In [1]:
import random

In [57]:
def profile_matrix(motifs):
    k = len(motifs[0])
    profile = {'A': [0] * k, 'C': [0] * k, 'G': [0] * k, 'T': [0] * k}
    for motif in motifs:
        for i in range(k):
            profile[motif[i]][i] += 1
    for base in profile:
        for i in range(k):
            profile[base][i] /= len(motifs)
    return profile

def profileMatrix(motifs, pseudocounts=1):
    profile = {}
    A, C, G, T = [], [], [], []
    for number in range(len(motifs[0])):
        countA, countC, countG, countT = pseudocounts, pseudocounts, pseudocounts, pseudocounts
        for motif in motifs:
            if motif[number] == "A":
                countA += 1
            elif motif[number] == "C":
                countC += 1
            elif motif[number] == "G":
                countG += 1
            elif motif[number] == "T":
                countT += 1
        A.append(countA / (len(motifs) + 4 * pseudocounts))
        C.append(countC / (len(motifs) + 4 * pseudocounts))
        G.append(countG / (len(motifs) + 4 * pseudocounts))
        T.append(countT / (len(motifs) + 4 * pseudocounts))
    profile["A"] = A
    profile["C"] = C
    profile["G"] = G
    profile["T"] = T
    return profile


def findProfileMostProbableKmer(string, k, matrix):
    seq = {}
    for i in range(len(string) - k + 1):
        seq[string[i:(i+k)]] = probableKmer(string[i:i +k], matrix)
    max_key = sorted(seq.items(), key=lambda x:x[1], reverse=True)[0][0]
    return max_key

def probableKmer(string, matrix):
    probable = 1
    for i in range(len(string)):
        if string[i] == 'A':
            probable = probable * matrix['A'][i]
    for i in range(len(string)):
        if string[i] == 'C':
            probable = probable * matrix['C'][i]
    for i in range(len(string)):
        if string[i] == 'G':
            probable = probable * matrix['G'][i]
    for i in range(len(string)):
        if string[i] == 'T':
            probable = probable * matrix['T'][i]
    return probable

def score(motifs):
    score = 0
    for i in range(len(motifs[0])):
        j = [motif[i] for motif in motifs]
        score += (len(j) - max(j.count("A"), j.count("C"), j.count("G"), j.count("T"),))
    return score

In [118]:
def randomize(DNA, k):
    random_kmers = []
    for dna_strand in DNA:
        random_number = random.randint(0,(len(DNA[0])-k))
        random_kmers.append(dna_strand[random_number:(random_number+k)])
    return random_kmers

In [267]:
def randomizedMotifSearch(DNA, k, t):
    motifs = randomize(DNA, k)
    # populating the best motifs with random k_mers
    best_motifs = motifs
    count = 0
    while True:
        profile = profileMatrix(motifs)
        motifs = []
        for dna_string in dna_list:
            motif = findProfileMostProbableKmer(dna_string, k, profile)
            motifs.append(motif)
        if score(motifs) < score(best_motifs):
            best_motifs = motifs
            count += 1
        else:
            return best_motifs

In [268]:
with open('rosalind_ba2a.txt', 'r') as f:
    full_list = [line.strip() for line in f]
    first_line_split = full_list[0].split()
    dna_list = []
    for number in range(1,len(full_list)):
        dna_list.append(full_list[number])
    k_value = int(first_line_split[0])
    t_value = int(first_line_split[1])

In [269]:
randomizedMotifSearch(dna_list, k_value, t_value)

['AACGGCCA', 'TAAGTGCC', 'TAGTACCG', 'CACGTCGG', 'CACGTGCA']

In [272]:
best_motifs = randomizedMotifSearch(dna_list, k_value, t_value)
for index in range(1000):
    motifs = randomizedMotifSearch(dna_list, k_value, t_value)
    if score(motifs) < score(best_motifs):
        best_motifs = motifs
for motif in motifs:
    print(motif)

TCAGTAAA
GTGCCAAG
CCGAAAGA
ACGTCGGT
CCACCAGC


In [187]:
import numpy as np

def read_data(file_name):
    with open(file_name, 'r') as file:
        k, t = (int(element) for element in (file.readline().strip()).split())
        motifs = [motif.strip() for motif in file]

    return k, t, motifs

def rand_motif_search(dna, k, t):
    last_ind = len(dna[0]) - k + 1
    idxs = [np.random.randint(0, last_ind) for _ in range(t)]
    best_motifs = [s[start:(start+k)] for s, start in zip(dna, idxs)]

    while True:
        profile = create_profile(best_motifs)
        motifs = create_motifs(profile, dna)
        if score(motifs) < score(best_motifs):
            best_motifs = motifs
        else:
            return best_motifs

def create_profile(motifs):
    profile = dict()
    motif_len = len(motifs[0])
    motifs_num = len(motifs)
    profile['A'] = [1 for _ in range(motif_len)]
    profile['C'] = [1 for _ in range(motif_len)]
    profile['G'] = [1 for _ in range(motif_len)]
    profile['T'] = [1 for _ in range(motif_len)]

    for i in range(motifs_num):
        for j in range(motif_len):
            profile[motifs[i][j]][j] += 1

    return profile

def create_motifs(profile, dna):
    return [get_best_match(profile, s) for s in dna]

def get_best_match(profile, string):
    motif_len = len(profile['A'])
    scores = calc_prob_for_all(profile, string)
    start = scores.index(max(scores))
    return string[start:start + motif_len]

def calc_prob_for_all(profile, string):
    return [calc_prob(profile, string, pos) for pos in range(len(string)-len(profile['A']))]

def calc_prob(profile, string, pos):
    result = 1
    for i in range(len(profile['A'])):
        result *= profile[string[pos+i]][i]

    return result

def score(motifs):
    consensus = get_consensus(motifs)
    score = 0
    for motif in motifs:
        score += hamming_dist(consensus, motif)

    return score

def get_consensus(motifs):
    length = len(motifs[0])
    profile = dict()
    profile['A'] = [0 for _ in range(length)]
    profile['C'] = [0 for _ in range(length)]
    profile['G'] = [0 for _ in range(length)]
    profile['T'] = [0 for _ in range(length)]

    for i in range(len(motifs)):
        for j in range(length):
            profile[motifs[i][j]][j] += 1

    consensus = []
    for j in range(length):
        max_elem = max(profile['A'][j],
                         profile['C'][j],
                         profile['G'][j],
                         profile['T'][j])

        if max_elem == profile['A'][j]:
            consensus.append("A")
        elif max_elem == profile['C'][j]:
            consensus.append("C")
        elif max_elem == profile['G'][j]:
            consensus.append("G")
        else:
            consensus.append("T")

    return "".join(consensus)

def hamming_dist(str_one, str_two):
    """ returns number of hamming_dist between two strings """

    len_one = len(str_one)
    len_two = len(str_two)
    if len_one != len_two:
        raise ValueError("Strings have different lengths.")

    mismatches = 0
    for i in range(len_one):
        if str_one[i] != str_two[i]:
            mismatches += 1

    return mismatches

In [199]:
best_motifs = rand_motif_search(dna_list, k_value, t_value)
for index in range(1000):
    motifs = rand_motif_search(dna_list, k_value, t_value)
    if score(motifs) < score(best_motifs):
        best_motifs = motifs

for motif in motifs:
    print(motif)

GGGTGTTC
AAGGTGCC
AAGTATAC
AAGTTTCA
AATCCACC
